In [1]:
from abc import ABC, abstractmethod
from typing import List
from typing import List, Type, Tuple, Union

import numpy as np
import string
import json

## Models

In [2]:
class AbstractModel(ABC):
    def __init__(self, words: List[str], docs: List[List[str]]) -> None:
        self.words = words
        self.docs = docs

    @abstractmethod
    def build(self) -> np.ndarray:
        pass


class TermCountModel(AbstractModel):
    def build(self):
        model = np.zeros((len(self.words), len(self.docs)), dtype=int)

        for i, word in enumerate(self.words):
            for j, doc in enumerate(self.docs):
                model[i, j] = doc.count(word)

        return model


class TFIDFModel(TermCountModel):
    def build(self):
        term_count_model = super().build()
        model = np.zeros((len(self.words), len(self.docs)), dtype=float)

        for i, word in enumerate(self.words):
            for j, doc in enumerate(self.docs):
                tf = term_count_model[i, j] / len(doc)
                idf = np.log(sum(term_count_model[i] > 0))
                model[i, j] = tf * idf

        return model

## LSI

In [3]:
class LSI:
    """Latent Semantic Indexing.
    """

    def __init__(self, docs: List[str], query: str, model: Type[AbstractModel] = TFIDFModel,
                 rank_approximation: int = 2, stopwords: List[str] = None,
                 ignore_chars=string.punctuation) -> None:
        if stopwords is None:
            stopwords = []
        self.stopwords = stopwords
        self.ignore_chars = ignore_chars
        self.docs = list(map(self._parse, docs))
        self.words = self._get_words()
        self.query = self._parse_query(query)
        self.model = model
        self.rank_approximation = rank_approximation
        self.term_doc_matrix = self._build_term_doc_matrix()

    def _parse(self, text: str) -> List[str]:
        translator = str.maketrans(self.ignore_chars, ' ' * len(self.ignore_chars))
        return list(map(str.lower,
                        filter(lambda w: w not in self.stopwords,
                               text.translate(translator).split())))

    def _parse_query(self, query: str) -> np.ndarray:
        result = np.zeros(len(self.words))

        i = 0
        for word in sorted(self._parse(query)):
            while word > self.words[i]:
                i += 1
            if word == self.words[i]:
                result[i] += 1

        return result

    def _get_words(self) -> List[str]:
        words = set()

        for doc in self.docs:
            words = words | set(doc)

        return sorted(words)

    def _build_term_doc_matrix(self) -> np.ndarray:
        model = self.model(self.words, self.docs)
        np.savetxt('matrxi_A.txt', model.build(), delimiter=',', fmt='%f')
        return model.build()

    def _svd_with_dimensionality_reduction(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        u, s, v = np.linalg.svd(self.term_doc_matrix)
        s = np.diag(s)
        k = self.rank_approximation
        return u[:, :k], s[:k, :k], v[:, :k]

    def process(self) -> Tuple[np.ndarray, np.ndarray]:
        u_k, s_k, v_k = self._svd_with_dimensionality_reduction()

        q = self.query.T @ u_k @ np.linalg.pinv(s_k)
        d = self.term_doc_matrix.T @ u_k @ np.linalg.pinv(s_k)

        res = np.apply_along_axis(lambda row: self._sim(q, row), axis=1, arr=d)
        ranking = np.argsort(-res) + 1
        return ranking, res

    @staticmethod
    def _sim(x: np.ndarray, y: np.ndarray):
        return (x @ y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [4]:
articles_data = json.load(open('articles_porter_mystem.json'))['issue']['articles']
documents = []
documents_names = []
for article in articles_data:
    documents_names.append(article['link'])
    documents.append(article['annotation']['porter'])

In [5]:
def rank(query):
    lsi = LSI(documents, query)
    ranks = lsi.process()
    res = list(zip(ranks[1], documents_names))
    res.sort(reverse=True)
    return res

In [6]:
query = 'сследова динамик множеств критическ точек'

In [7]:
rank(query)

[(0.98694977435528541, 'http://www.mathnet.ru/rus/uzku1348'),
 (0.93780932622811231, 'http://www.mathnet.ru/rus/uzku1354'),
 (0.48310905675650379, 'http://www.mathnet.ru/rus/uzku1356'),
 (0.31724583956343499, 'http://www.mathnet.ru/rus/uzku1350'),
 (0.2137728675540487, 'http://www.mathnet.ru/rus/uzku1352'),
 (0.20875414777511714, 'http://www.mathnet.ru/rus/uzku1357'),
 (0.0036627067108410357, 'http://www.mathnet.ru/rus/uzku1358'),
 (-0.28309867572666014, 'http://www.mathnet.ru/rus/uzku1355'),
 (-0.2873375777621997, 'http://www.mathnet.ru/rus/uzku1349'),
 (-0.65817914190981652, 'http://www.mathnet.ru/rus/uzku1353')]

In [8]:
query = 'реш нестационарн задач для тонк упруг'

In [9]:
rank(query)

[(0.97199683499449663, 'http://www.mathnet.ru/rus/uzku1353'),
 (0.78743242374442557, 'http://www.mathnet.ru/rus/uzku1349'),
 (0.78469863059935585, 'http://www.mathnet.ru/rus/uzku1355'),
 (0.57411566265432312, 'http://www.mathnet.ru/rus/uzku1358'),
 (0.39391353117858835, 'http://www.mathnet.ru/rus/uzku1357'),
 (0.3891888872469681, 'http://www.mathnet.ru/rus/uzku1352'),
 (0.28821526498002059, 'http://www.mathnet.ru/rus/uzku1350'),
 (0.1107564954954537, 'http://www.mathnet.ru/rus/uzku1356'),
 (-0.56553245036212807, 'http://www.mathnet.ru/rus/uzku1354'),
 (-0.71307701120388933, 'http://www.mathnet.ru/rus/uzku1348')]

In [10]:
query = 'стабилизац решен дифференциальн уравнен в гильбертов пространств'

In [11]:
rank(query)

[(0.99103685277467801, 'http://www.mathnet.ru/rus/uzku1356'),
 (0.950682422308185, 'http://www.mathnet.ru/rus/uzku1350'),
 (0.91196167935246331, 'http://www.mathnet.ru/rus/uzku1352'),
 (0.90984305928194509, 'http://www.mathnet.ru/rus/uzku1357'),
 (0.83751681295668767, 'http://www.mathnet.ru/rus/uzku1354'),
 (0.80535111955725702, 'http://www.mathnet.ru/rus/uzku1358'),
 (0.71730317889866424, 'http://www.mathnet.ru/rus/uzku1348'),
 (0.60166293446289965, 'http://www.mathnet.ru/rus/uzku1355'),
 (0.59812450174363663, 'http://www.mathnet.ru/rus/uzku1349'),
 (0.21257282568923108, 'http://www.mathnet.ru/rus/uzku1353')]